In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from paths import mkl_data
import requests # API library

import random
import time
import os

from isoweek import Week

import requests # API library

import numpy as np
import pandas as pd
import json
import re

import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam
from pyro.infer import Predictive

import torch

# Pyro-specific imports
import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam


/var/folders/bx/n7x8tlw17p9_m_8d4g08505w0000gn/T/ipykernel_3281/2582778771.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/Users/fluffysnail/Library/CloudStorage/OneDrive-Personal/dtu/current/42186_modelml/42186_venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df_m1 = pd.read_csv('../df_matches_clean.csv')

In [5]:
#Toy Model - we assume that all skills are normal distributed with mean 0 and std 1
def simple_model(obs=None):
    skill1 = pyro.sample("skill1", dist.Normal(0., 1.))
    skill2 = pyro.sample("skill2", dist.Normal(0., 1.))

    sigma1 = 1; sigma2 = 2
    perf1 = pyro.sample("perf1", dist.Normal(skill1, sigma1))
    perf2 = pyro.sample("perf2", dist.Normal(skill2, sigma2))

    with pyro.plate("data", len(obs)):
        y = pyro.sample("obs", dist.Categorical(), obs=obs)
    
    return y

In [13]:
import torch
import pyro
import pyro.distributions as dist

def simple_model2(obs=None):
    mu1 = pyro.sample("mu1", dist.Normal(0., 1.))
    mu2 = pyro.sample("mu2", dist.Normal(0., 1.))

    sigma1 = pyro.sample("sigma1", dist.HalfNormal(1.))
    sigma2 = pyro.sample("sigma2", dist.HalfNormal(1.))
    
    skill1 = pyro.sample("skill1", dist.Normal(mu1, sigma1))
    skill2 = pyro.sample("skill2", dist.Normal(mu2, sigma2))

    sigma1 = 1
    sigma2 = 2
    perf1 = pyro.sample("perf1", dist.Normal(skill1, sigma1))
    perf2 = pyro.sample("perf2", dist.Normal(skill2, sigma2))

    # Calculate performance difference
    perf_diff = perf1 - perf2

    # Translate performance difference into a vector of outcome probabilities
    # The scaling factor 'scale' can be adjusted to control the spread
    scale = torch.tensor(1.0)
    logits = torch.tensor([-1., 0., 1.]) * perf_diff * scale
    probabilities = torch.softmax(logits, 0)

    with pyro.plate("data", len(obs) if obs is not None else 1):
        # Map observed outcomes to indices for the Categorical distribution
        if obs is not None:
            obs_indices = (torch.tensor(obs) + 1).long()
        else:
            obs_indices = None

        # Sample the observation or score it against the model
        cat = pyro.sample("obs", dist.Categorical(probabilities), obs=obs_indices)
        
        # If we are not conditioning on observed data, map the sample back to [-1, 0, 1]
        y = torch.tensor([-1, 0, 1])[cat.long()] if obs is None else obs

    return y


In [15]:
# pyro.render_model(model, model_args=(None, None), render_distributions=True)

#Make guide and optimizer
model = simple_model2
auto_guide = pyro.infer.autoguide.AutoNormal(model)
adam = pyro.optim.Adam({"lr": 0.02})
elbo = pyro.infer.Trace_ELBO()
svi = pyro.infer.SVI(model, auto_guide, adam, elbo)

In [16]:
%%time

#Clear param store 
pyro.clear_param_store()

# These should be reset each training loop.
auto_guide = pyro.infer.autoguide.AutoNormal(model)
adam = pyro.optim.Adam({"lr": 0.005}) 
elbo = pyro.infer.Trace_ELBO(num_particles=3)
svi = pyro.infer.SVI(model, auto_guide, adam, elbo)


# Do gradient steps
n_steps = 1000
X_torch = torch.tensor(df_m1['team1_win'].values, dtype=torch.int32)
for step in range(n_steps):
    elbo = svi.step(X_torch)
    if step % 100 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

/var/folders/bx/n7x8tlw17p9_m_8d4g08505w0000gn/T/ipykernel_3281/2858623440.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  obs_indices = (torch.tensor(obs) + 1).long()


[0] ELBO: 105210.4


/var/folders/bx/n7x8tlw17p9_m_8d4g08505w0000gn/T/ipykernel_3281/2858623440.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  obs_indices = (torch.tensor(obs) + 1).long()


[100] ELBO: 104586.8
[200] ELBO: 104849.0
[300] ELBO: 104669.9
[400] ELBO: 104610.8
[500] ELBO: 104604.3
[600] ELBO: 104576.7
[700] ELBO: 104538.5
[800] ELBO: 104527.2
[900] ELBO: 104524.0
CPU times: user 19.9 s, sys: 58.5 s, total: 1min 18s
Wall time: 9.78 s
